In [40]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
import scipy as sp
from scipy.stats import boxcox
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [41]:
#2013 data
ny = pd.read_excel("/Users/bradleyrhyins/Desktop/Thinkful-Projects/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls", sep=r'\\t')
nj = pd.read_excel("/Users/bradleyrhyins/Downloads/table_8_offenses_known_to_law_enforcement_new_jersey_by_city_2013.xls", sep=r'\\t')
ct = pd.read_excel("/Users/bradleyrhyins/Downloads/table_8_offenses_known_to_law_enforcement_connecticut_by_city_2013.xls", sep=r'\\t')

In [42]:
#2012 data
ny12 = pd.read_excel("/Users/bradleyrhyins/Downloads/table_8_offenses_known_to_law_enforcement_by_new_york_by_city_2012.xls", sep=r'\\t')
nj12 = pd.read_excel("/Users/bradleyrhyins/Downloads/table_8_offenses_known_to_law_enforcement_by_new_jersey_by_city_2012.xls", sep=r'\\t')
ct12 = pd.read_excel("/Users/bradleyrhyins/Downloads/table_8_offenses_known_to_law_enforcement_by_connecticut_by_city_2012.xls", sep=r'\\t')

In [43]:
#2014 data
ny14 = pd.read_excel("/Users/bradleyrhyins/Downloads/Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.xls", sep=r'\\t')
nj14 = pd.read_excel("/Users/bradleyrhyins/Downloads/Table_8_Offenses_Known_to_Law_Enforcement_by_New_Jersey_by_City_2014.xls", sep=r'\\t')
ct14 = pd.read_excel("/Users/bradleyrhyins/Downloads/Table_8_Offenses_Known_to_Law_Enforcement_by_Connecticut_by_City_2014.xls", sep=r'\\t')

In [51]:
#get rid of nans
nj.fillna(0, inplace=True)

In [62]:
nj.columns

Index([                        u'City',                   u'Population',
                     u'Violent\ncrime',                       u'Murder',
       u'Rape\n(revised\ndefinition)1',  u'Rape\n(legacy\ndefinition)2',
                            u'Robbery',          u'Aggravated\nassault',
                    u'Property\ncrime',                     u'Burglary',
                    u'Larceny-\ntheft',        u'Motor\nvehicle\ntheft',
                              u'Arson',                u'Propertycrime',
                     u'MurderBurglary'],
      dtype='object')

In [63]:
# Acquire, load, and preview the data.
display(nj.head())

,City,Population,Violent crime,Murder,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Propertycrime,MurderBurglary
0,Aberdeen Township,18150.000,19.000,0.000,0.000,0.000,1,6.000,237.000,1,185.000,10.000,0.000,0,1
1,Absecon,8380.000,21.000,0.000,0.000,2.000,1,15.000,266.000,1,169.000,6.000,0.000,0,1
2,Allendale,6712.000,0.000,0.000,0.000,0.000,0,0.000,37.000,1,29.000,2.000,0.000,0,0
3,Allenhurst,493.000,0.000,0.000,0.000,0.000,0,0.000,39.000,1,18.000,1.000,0.000,0,0
4,Allentown,1812.000,3.000,0.000,0.000,0.000,0,3.000,18.000,1,6.000,0.000,0.000,0,0


In [64]:
nj["Robbery"] = [1 if x > 0 else 0 for x in nj["Robbery"]]
nj["Propertycrime"] = [1 if x > 0 else 0 for x in nj["Murder"]]
nj["Burglary"] = [1 if x > 0 else 0 for x in nj["Burglary"]]

In [65]:
print nj.head()

                City  Population  Violent\ncrime  Murder  \
0  Aberdeen Township   18150.000          19.000   0.000   
1            Absecon    8380.000          21.000   0.000   
2          Allendale    6712.000           0.000   0.000   
3         Allenhurst     493.000           0.000   0.000   
4          Allentown    1812.000           3.000   0.000   

   Rape\n(revised\ndefinition)1  Rape\n(legacy\ndefinition)2  Robbery  \
0                         0.000                        0.000        1   
1                         0.000                        2.000        1   
2                         0.000                        0.000        0   
3                         0.000                        0.000        0   
4                         0.000                        0.000        0   

   Aggravated\nassault  Property\ncrime  Burglary  Larceny-\ntheft  \
0                6.000          237.000         1          185.000   
1               15.000          266.000         1          1

In [66]:
nj["MurderBurglary"] = [x*y for x, y in zip(nj["Robbery"], nj["Burglary"])]

In [67]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
#use this format for formatting sample examples in course material
Y = nj['Propertycrime'].values.reshape(-1, 1)
X = nj[['Population','Robbery','Burglary','MurderBurglary']]
regr.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [68]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

('\nCoefficients: \n', array([[ 6.68537940e-06, -2.73373520e-02,  7.78211023e-03,
         1.11881034e-01]]))
('\nIntercept: \n', array([-0.00554803]))

R-squared:
0.2377019535450201


In [69]:
nj["logPop"] = [0 if x == 0 else np.log(x) for x in nj["Population"]]
nj["logRob"] = [0 if x == 0 else np.log(x) for x in nj["Robbery"]]
nj["logBur"] = [0 if x == 0 else np.log(x) for x in nj["Burglary"]]
nj["logMB"] = [0 if x == 0 else np.log(x) for x in nj["MurderBurglary"]]

In [ ]:
# Extract predicted values.
predicted = regr.predict(X).ravel()
actual = data['Sales']

# Calculate the error, also called the residual.
residual = actual - predicted

# This looks a bit concerning.
plt.hist(residual)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
plt.show()